# 8 PUZZLE PROBLEM

In [0]:
import argparse
import timeit
import resource
from collections import deque
from heapq import heappush, heappop, heapify
import itertools
import csv

In [0]:
class State:

    def __init__(self, state, parent, move, depth, cost, key):

        self.state = state

        self.parent = parent

        self.move = move

        self.depth = depth

        self.cost = cost

        self.key = key

        if self.state:
            self.map = ''.join(str(e) for e in self.state)

    def __eq__(self, other):
        return self.map == other.map

    def __lt__(self, other):
        return self.map < other.map

In [0]:
goal_state = [0, 1, 2, 3, 4, 5, 6, 7, 8]
goal_node = State
initial_state = list()
board_len = 0
board_side = 0

nodes_expanded = 0
max_search_depth = 0
max_frontier_size = 0

moves = list()
costs = set()

# BFS

In [0]:
def bfs(start_state):

    global max_frontier_size, goal_node, max_search_depth

    explored, queue = set(), deque([State(start_state, None, None, 0, 0, 0)])

    while queue:

        node = queue.popleft()

        explored.add(node.map)

        if node.state == goal_state:
            goal_node = node
            return queue

        neighbors = expand(node)

        for neighbor in neighbors:
            if neighbor.map not in explored:
                queue.append(neighbor)
                explored.add(neighbor.map)

                if neighbor.depth > max_search_depth:
                    max_search_depth += 1

        if len(queue) > max_frontier_size:
            max_frontier_size = len(queue)


# DFS

In [0]:
def dfs(start_state):

    global max_frontier_size, goal_node, max_search_depth

    explored, stack = set(), list([State(start_state, None, None, 0, 0, 0)])

    while stack:

        node = stack.pop()

        explored.add(node.map)

        if node.state == goal_state:
            goal_node = node
            return stack

        neighbors = reversed(expand(node))

        for neighbor in neighbors:
            if neighbor.map not in explored:
                stack.append(neighbor)
                explored.add(neighbor.map)

                if neighbor.depth > max_search_depth:
                    max_search_depth += 1

        if len(stack) > max_frontier_size:
            max_frontier_size = len(stack)

# A*

In [0]:
def ast(start_state):

    global max_frontier_size, goal_node, max_search_depth

    explored, heap, heap_entry, counter = set(), list(), {}, itertools.count()

    key = h(start_state)

    root = State(start_state, None, None, 0, 0, key)

    entry = (key, 0, root)

    heappush(heap, entry)

    heap_entry[root.map] = entry

    while heap:

        node = heappop(heap)

        explored.add(node[2].map)

        if node[2].state == goal_state:
            goal_node = node[2]
            return heap

        neighbors = expand(node[2])

        for neighbor in neighbors:

            neighbor.key = neighbor.cost + h(neighbor.state)

            entry = (neighbor.key, neighbor.move, neighbor)

            if neighbor.map not in explored:

                heappush(heap, entry)

                explored.add(neighbor.map)

                heap_entry[neighbor.map] = entry

                if neighbor.depth > max_search_depth:
                    max_search_depth += 1

            elif neighbor.map in heap_entry and neighbor.key < heap_entry[neighbor.map][2].key:

                hindex = heap.index((heap_entry[neighbor.map][2].key,
                                     heap_entry[neighbor.map][2].move,
                                     heap_entry[neighbor.map][2]))

                heap[int(hindex)] = entry

                heap_entry[neighbor.map] = entry

                heapify(heap)

        if len(heap) > max_frontier_size:
            max_frontier_size = len(heap)

# IDA

In [0]:
def ida(start_state):

    global costs

    threshold = h(start_state)

    while 1:
        response = dlss_mod(start_state, threshold)

        if type(response) is list:
            return response
            break

        threshold = response

        costs = set()

In [0]:
def dlss_mod(start_state, threshold):

    global max_frontier_size, goal_node, max_search_depth, costs

    explored, stack = set(), list([State(start_state, None, None, 0, 0, threshold)])

    while stack:

        node = stack.pop()

        explored.add(node.map)

        if node.state == goal_state:
            goal_node = node
            return stack

        if node.key > threshold:
            costs.add(node.key)

        if node.depth < threshold:

            neighbors = reversed(expand(node))

            for neighbor in neighbors:
                if neighbor.map not in explored:

                    neighbor.key = neighbor.cost + h(neighbor.state)
                    stack.append(neighbor)
                    explored.add(neighbor.map)

                    if neighbor.depth > max_search_depth:
                        max_search_depth += 1

            if len(stack) > max_frontier_size:
                max_frontier_size = len(stack)

    return min(costs)


In [0]:
def expand(node):

    global nodes_expanded
    nodes_expanded += 1

    neighbors = list()

    neighbors.append(State(move(node.state, 1), node, 1, node.depth + 1, node.cost + 1, 0))
    neighbors.append(State(move(node.state, 2), node, 2, node.depth + 1, node.cost + 1, 0))
    neighbors.append(State(move(node.state, 3), node, 3, node.depth + 1, node.cost + 1, 0))
    neighbors.append(State(move(node.state, 4), node, 4, node.depth + 1, node.cost + 1, 0))

    nodes = [neighbor for neighbor in neighbors if neighbor.state]

    return nodes

In [0]:
def move(state, position):

    new_state = state[:]

    index = new_state.index(0)

    if position == 1:  # Up

        if index not in range(0, board_side):

            temp = new_state[index - board_side]
            new_state[index - board_side] = new_state[index]
            new_state[index] = temp

            return new_state
        else:
            return None

    if position == 2:  # Down

        if index not in range(board_len - board_side, board_len):

            temp = new_state[index + board_side]
            new_state[index + board_side] = new_state[index]
            new_state[index] = temp

            return new_state
        else:
            return None

    if position == 3:  # Left

        if index not in range(0, board_len, board_side):

            temp = new_state[index - 1]
            new_state[index - 1] = new_state[index]
            new_state[index] = temp

            return new_state
        else:
            return None

    if position == 4:  # Right

        if index not in range(board_side - 1, board_len, board_side):

            temp = new_state[index + 1]
            new_state[index + 1] = new_state[index]
            new_state[index] = temp

            return new_state
        else:
            return None

In [0]:
def h(state):

    return sum(abs(b % board_side - g % board_side) + abs(b//board_side - g//board_side)
               for b, g in ((state.index(i), goal_state.index(i)) for i in range(1, board_len)))

In [0]:
def backtrace():

    current_node = goal_node

    while initial_state != current_node.state:

        if current_node.move == 1:
            movement = 'Up'
        elif current_node.move == 2:
            movement = 'Down'
        elif current_node.move == 3:
            movement = 'Left'
        else:
            movement = 'Right'

        moves.insert(0, movement)
        current_node = current_node.parent

    return moves

In [0]:
def export(algo, frontier, time, write_mode):

    global moves

    moves = backtrace()

    file = open('output.txt', write_mode)
    file.write("\nALGORITHM: " + algo)
    file.write("\npath_to_goal: " + str(moves))
    file.write("\ncost_of_path: " + str(len(moves)))
    file.write("\nnodes_expanded: " + str(nodes_expanded))
    file.write("\nfringe_size: " + str(len(frontier)))
    file.write("\nmax_fringe_size: " + str(max_frontier_size))
    file.write("\nsearch_depth: " + str(goal_node.depth))
    file.write("\nmax_search_depth: " + str(max_search_depth))
    file.write("\nrunning_time: " + format(time, '.8f'))
    file.write("\nmax_ram_usage: " + format(resource.getrusage(resource.RUSAGE_SELF).ru_maxrss/1000.0, '.8f'))    
    file.write("\n-----------------------------------------------------------------------------\n")
    file.close()
    
    with open('8-puzzle-results.csv', mode=write_mode) as file:
            writer = csv.writer(file, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
            if(write_mode == 'w'):
                writer.writerow(['algorithm', 'cost_of_path', 'nodes_expanded', 'search_depth', 'running_time', 'max_ram_usage'])
            writer.writerow([algo, len(moves), str(nodes_expanded), str(goal_node.depth),
                             format(time, '.8f'), format(resource.getrusage(resource.RUSAGE_SELF).ru_maxrss/1000.0, '.8f')])

In [0]:
def read(configuration):

    global board_len, board_side

    data = configuration.split(",")

    for element in data:
        initial_state.append(int(element))

    board_len = len(initial_state)

    board_side = int(board_len ** 0.5)

In [0]:
read('0,8,7,6,5,4,3,2,1')

In [38]:
start = timeit.default_timer()

frontier = bfs(initial_state)

stop = timeit.default_timer()

export("BFS", frontier, stop-start, 'w')

print("BFS execution complete!")

BFS execution complete!


In [39]:
start = timeit.default_timer()

frontier = dfs(initial_state)

stop = timeit.default_timer()

export("DFS", frontier, stop-start, 'a')

print("DFS execution complete!")

DFS execution complete!


In [40]:
start = timeit.default_timer()

frontier = ast(initial_state)

stop = timeit.default_timer()

export("A*", frontier, stop-start, 'a')

print("A* execution complete!")

A* execution complete!


In [41]:
start = timeit.default_timer()

frontier = ida(initial_state)

stop = timeit.default_timer()

export("IDA", frontier, stop-start, 'a')

print("IDA execution complete!")

IDA execution complete!


In [0]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go

In [0]:
eight_puzzle_results = pd.read_csv("8-puzzle-results.csv")

In [44]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=eight_puzzle_results['algorithm'], 
                     y=eight_puzzle_results['cost_of_path'],
                     name="Cost of Path" ))
fig.update_layout(title_text='Cost of Path')
fig.show()

In [45]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=eight_puzzle_results['algorithm'], 
                     y=eight_puzzle_results['search_depth'],
                     name="Search Depth" ))
fig.update_layout(title_text='Search Depth')
fig.show()

In [46]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=eight_puzzle_results['algorithm'], 
                     y=eight_puzzle_results['running_time'],
                     name="Running time" ))
fig.update_layout(title_text='Running time')
fig.show()